# 【案例12】Braneast 航空公司的航线安排——参考求解

In [47]:
# 定义节点之间的最大流量
import numpy as np
flow = np.zeros([3,3,14,14])
#i ,j,ts,te = 1,1,8,9
#print(flow[i-1][j-1][ts-8][te-8])

# 航线的最大流量是1
flow[0][1][10-8][12-8] = 1
flow[0][1][14-8][16-8] = 1
flow[0][2][11-8][14-8] = 1
flow[0][2][15-8][18-8] = 1
flow[1][0][10-8][12-8] = 1
flow[1][0][16-8][18-8] = 1
flow[1][2][9 -8][11-8] = 1
flow[1][2][14-8][16-8] = 1
flow[2][0][10-8][13-8] = 1
flow[2][0][17-8][20-8] = 1
flow[2][1][9 -8][11-8] = 1
flow[2][1][14-8][16-8] = 1
# 在机场内部的最大流量没有限制flow[i,i,t,t+1] = BIGM
BIGM = 9999996
for i in range(0,3):
    for t in range(9,19):
        flow[i][i][t-8][t+1 - 8] = BIGM
flow[0][0][19-8][20-8] = BIGM
# 从起始节点S出发去往各个节点的最大流量没有限制
BIGM = 10000000
flow[0][0][8-8][9-8] = BIGM
flow[0][1][8-8][9-8] = BIGM
flow[0][2][8-8][9-8] = BIGM
# 到达结束节点E的边的最大流量没有限制
flow[0][0][20-8][21-8] = BIGM
flow[1][0][19-8][21-8] = BIGM
flow[2][0][19-8][21-8] = BIGM
# 其他边的最大流量为0（即这些边不存在）

# 定义各条边的利润/成本
revenue = np.zeros([3,3,14,14])
# 各条航线的利润
revenue[0][1][10-8][12-8] = 900-500
revenue[0][1][14-8][16-8] = 800-450
revenue[0][2][11-8][14-8] = 1100-600
revenue[0][2][15-8][18-8] = 1200-650
revenue[1][0][10-8][12-8] = 800-400
revenue[1][0][16-8][18-8] = 1200-450
revenue[1][2][9 -8][11-8] = 900-400
revenue[1][2][14-8][16-8] = 600-350
revenue[2][0][10-8][13-8] = 1500-700
revenue[2][0][17-8][20-8] = 1800-900
revenue[2][1][9 -8][11-8] = 1100-400
revenue[2][1][14-8][16-8] = 900-350

# for i in range(1,3+1):
#     for ts in range(8,22):
#         for te in range(8,22):
#             for j in range(1,3+1):     
#                 if flow[i-1][j-1][ts-8][te-8] >0:
#                     print(f"city {i} -> city {j} ,time :{ts}->{te},\tRevenue = {revenue[i-1][j-1][ts-8][te-8]}")

## 1.最小的飞机数量

In [48]:
# 定义线性规划问题
from pulp import *
prob = LpProblem("Airlines", LpMinimize)
# 定义变量
f = LpVariable.dicts("f",[(i,j,ts,te) for i in range(1,3+1) for j in range(1,3+1) for ts in range(8,22) for te in range(8,22)],lowBound=0,cat='Integer')
TotalAirplane = LpVariable("TotalAirplane", lowBound=0, cat='Integer') 
TotalRevenue = LpVariable("TotalRevenue",cat='Continuous')

# 定义目标函数
prob += TotalAirplane

# 定义约束条件
prob += TotalAirplane == lpSum(f[1,j,8,9] for j in range(1,3+1))
prob += TotalRevenue == lpSum(f[i,j,ts,te] * revenue[i-1][j-1][ts-8][te-8] for i in range(1,3+1) for j in range(1,3+1) for ts in range(8,22) for te in range(8,22) if flow[i-1][j-1][ts-8][te-8] > 0 )

# 边<i,j,ts,te>的流量不超过最大流量限制:
for i in range(1,3+1):
    for j in range(1,3+1):
        for ts in range(8,21):
            for te in range(8,21):
                prob += f[i,j,ts,te] <= flow[i-1][j-1][ts-8][te-8]
                prob += f[i,j,ts,te] >= 0
                # 如果<i,j,ts,te>执行的是航线而不是在机场里面走
                if flow[i-1][j-1][ts-8][te-8] == 1:
                    prob += f[i,j,ts,te] == flow[i-1][j-1][ts-8][te-8]
                    
                
# 除起始节点S和结束节点E外，其他的每个节点node[i,ts]的流入流量=流出流量 
for i in range(1,3+1):
    for ti in range(9,21):
        prob += lpSum(f[h,i,th,ti] for h in range(1,4) for th in range(8,21)) == lpSum(f[i,j,ti,tj] for j in range(1,4) for tj in range(9,22) )

# 起始节点S的流出流量=结束节点的流入流量
prob += lpSum(f[1,j,8,9] for j in range(1,3)) == f[1,1,20,21] + f[2,1,19,21] + f[3,1,19,21]

prob.solve()
if LpStatus[prob.status] == "Optimal":  
    print("找到最优解")
else:
    print("No solution")

找到最优解


In [49]:
print(value(TotalAirplane))

6.0


In [50]:
# 查看所有航线是否都正常运行
for i in range(1,4):
    for j in range(1,4):
        for ts in range(9,20):
            for te in range(9,21):
                if flow[i-1][j-1][ts-8][te-8] == 1:
                    status = "OK"
                    if value(f[i,j,ts,te]) == 0:
                        status = "Canceled"
                    print(f"Airline \t{city[i-1]} -> \t{city[j-1]} \tdepart at {int(ts)}:00  \t{status}"  )

Airline 	波士顿 -> 	纽约 	depart at 10:00  	OK
Airline 	波士顿 -> 	纽约 	depart at 14:00  	OK
Airline 	波士顿 -> 	华盛顿 	depart at 11:00  	OK
Airline 	波士顿 -> 	华盛顿 	depart at 15:00  	OK
Airline 	纽约 -> 	波士顿 	depart at 10:00  	OK
Airline 	纽约 -> 	波士顿 	depart at 16:00  	OK
Airline 	纽约 -> 	华盛顿 	depart at 9:00  	OK
Airline 	纽约 -> 	华盛顿 	depart at 14:00  	OK
Airline 	华盛顿 -> 	波士顿 	depart at 10:00  	OK
Airline 	华盛顿 -> 	波士顿 	depart at 17:00  	OK
Airline 	华盛顿 -> 	纽约 	depart at 9:00  	OK
Airline 	华盛顿 -> 	纽约 	depart at 14:00  	OK


## 2. 最小的运营成本

In [51]:
# 定义线性规划问题
from pulp import *
prob = LpProblem("Airlines2", LpMaximize)
# 定义变量
f = LpVariable.dicts("f",[(i,j,ts,te) for i in range(1,3+1) for j in range(1,3+1) for ts in range(8,22) for te in range(8,22)],lowBound=0,cat='Integer')
TotalAirplane = LpVariable("TotalAirplane", lowBound=0, cat='Integer') 
TotalRevenue = LpVariable("TotalRevenue",cat='Continuous')

# 定义目标函数
prob += TotalRevenue

# 定义约束条件
prob += TotalAirplane == lpSum(f[1,j,8,9] for j in range(1,3+1))
prob += TotalRevenue == lpSum(f[i,j,ts,te] * revenue[i-1][j-1][ts-8][te-8] for i in range(1,3+1) for j in range(1,3+1) for ts in range(8,22) for te in range(8,22) if flow[i-1][j-1][ts-8][te-8] > 0 )
prob += TotalAirplane == 4

# 边<i,j,ts,te>的流量不超过最大流量限制:
for i in range(1,3+1):
    for j in range(1,3+1):
        for ts in range(8,21):
            for te in range(8,21):
                prob += f[i,j,ts,te] <= flow[i-1][j-1][ts-8][te-8]
                prob += f[i,j,ts,te] >= 0
                    
                
# 除起始节点S和结束节点E外，其他的每个节点node[i,ts]的流入流量=流出流量 
for i in range(1,3+1):
    for ti in range(9,21):
        prob += lpSum(f[h,i,th,ti] for h in range(1,4) for th in range(8,21)) == lpSum(f[i,j,ti,tj] for j in range(1,4) for tj in range(9,22) )

# 起始节点S的流出流量=结束节点的流入流量
prob += lpSum(f[1,j,8,9] for j in range(1,3)) == f[1,1,20,21] + f[2,1,19,21] + f[3,1,19,21]

prob.solve()
if LpStatus[prob.status] == "Optimal":  
    print("找到最优解")
else:
    print("No solution")

找到最优解


In [53]:
print(value(TotalRevenue))

5750.0


In [54]:
# 查看正常运行的航线
city = ["波士顿","纽约","华盛顿"]
for i in range(1,4):
    for j in range(1,4):
        for ts in range(9,20):
            for te in range(9,21):
                if flow[i-1][j-1][ts-8][te-8] == 1:
                    status = "OK"
                    if value(f[i,j,ts,te]) == 0:
                        status = "Canceled"
                    print(f"Airline \t{city[i-1]} -> \t{city[j-1]} \tdepart at {int(ts)}:00  \t{status}"  )
                

Airline 	波士顿 -> 	纽约 	depart at 10:00  	Canceled
Airline 	波士顿 -> 	纽约 	depart at 14:00  	OK
Airline 	波士顿 -> 	华盛顿 	depart at 11:00  	OK
Airline 	波士顿 -> 	华盛顿 	depart at 15:00  	OK
Airline 	纽约 -> 	波士顿 	depart at 10:00  	OK
Airline 	纽约 -> 	波士顿 	depart at 16:00  	OK
Airline 	纽约 -> 	华盛顿 	depart at 9:00  	Canceled
Airline 	纽约 -> 	华盛顿 	depart at 14:00  	OK
Airline 	华盛顿 -> 	波士顿 	depart at 10:00  	OK
Airline 	华盛顿 -> 	波士顿 	depart at 17:00  	OK
Airline 	华盛顿 -> 	纽约 	depart at 9:00  	OK
Airline 	华盛顿 -> 	纽约 	depart at 14:00  	OK


## 3.考虑飞机在机场的停机费用

In [55]:
# 在机场内部每小时的流动停机成本
REST_COST = 50
for i in range(0,3):
    for t in range(9,19):
        revenue[i][i][t-8][t+1 - 8] = -1*REST_COST
revenue[0][0][19-8][20-8] = -1*REST_COST

In [56]:
# 定义线性规划问题
from pulp import *
prob = LpProblem("Airlines2", LpMaximize)
# 定义变量
f = LpVariable.dicts("f",[(i,j,ts,te) for i in range(1,3+1) for j in range(1,3+1) for ts in range(8,22) for te in range(8,22)],lowBound=0,cat='Integer')
TotalAirplane = LpVariable("TotalAirplane", lowBound=0, cat='Integer') 
TotalRevenue = LpVariable("TotalRevenue",cat='Continuous')

# 定义目标函数
prob += TotalRevenue

# 定义约束条件
prob += TotalAirplane == lpSum(f[1,j,8,9] for j in range(1,3+1))
prob += TotalRevenue == lpSum(f[i,j,ts,te] * revenue[i-1][j-1][ts-8][te-8] for i in range(1,3+1) for j in range(1,3+1) for ts in range(8,22) for te in range(8,22) if flow[i-1][j-1][ts-8][te-8] > 0 )
prob += TotalAirplane == 4

# 边<i,j,ts,te>的流量不超过最大流量限制:
for i in range(1,3+1):
    for j in range(1,3+1):
        for ts in range(8,21):
            for te in range(8,21):
                prob += f[i,j,ts,te] <= flow[i-1][j-1][ts-8][te-8]
                prob += f[i,j,ts,te] >= 0
                    
                
# 除起始节点S和结束节点E外，其他的每个节点node[i,ts]的流入流量=流出流量 
for i in range(1,3+1):
    for ti in range(9,21):
        prob += lpSum(f[h,i,th,ti] for h in range(1,4) for th in range(8,21)) == lpSum(f[i,j,ti,tj] for j in range(1,4) for tj in range(9,22) )

# 起始节点S的流出流量=结束节点的流入流量
prob += lpSum(f[1,j,8,9] for j in range(1,3)) == f[1,1,20,21] + f[2,1,19,21] + f[3,1,19,21]

prob.solve()
if LpStatus[prob.status] == "Optimal":  
    print("找到最优解")
else:
    print("No solution")

找到最优解


In [57]:
print(value(TotalRevenue))

5100.0


In [59]:
# 查看正常运行的航线
city = ["波士顿","纽约","华盛顿"]
for i in range(1,4):
    for j in range(1,4):
        for ts in range(9,20):
            for te in range(9,21):
                if flow[i-1][j-1][ts-8][te-8] == 1:
                    status = "OK"
                    if value(f[i,j,ts,te]) == 0:
                        status = "Canceled"
                    print(f"Airline \t{city[i-1]} -> \t{city[j-1]} \tdepart at {int(ts)}:00  \t{status}"  )           

Airline 	波士顿 -> 	纽约 	depart at 10:00  	Canceled
Airline 	波士顿 -> 	纽约 	depart at 14:00  	OK
Airline 	波士顿 -> 	华盛顿 	depart at 11:00  	OK
Airline 	波士顿 -> 	华盛顿 	depart at 15:00  	OK
Airline 	纽约 -> 	波士顿 	depart at 10:00  	OK
Airline 	纽约 -> 	波士顿 	depart at 16:00  	OK
Airline 	纽约 -> 	华盛顿 	depart at 9:00  	Canceled
Airline 	纽约 -> 	华盛顿 	depart at 14:00  	OK
Airline 	华盛顿 -> 	波士顿 	depart at 10:00  	OK
Airline 	华盛顿 -> 	波士顿 	depart at 17:00  	OK
Airline 	华盛顿 -> 	纽约 	depart at 9:00  	OK
Airline 	华盛顿 -> 	纽约 	depart at 14:00  	OK


可以看到,当停机成本上升时，总利润下降了